In [1]:
import pandas as pd

# Read Excel file
df = pd.read_excel("wide_data.xlsx")

# Show first 5 rows
print(df.head())


   Id       Date  Rating  Year
0   1 2000-05-30       7  2000
1   1 2000-12-31       6  2000
2   2 2003-05-21       6  2003
3   3 1999-12-30       5  1999
4   3 2000-10-30       6  2000


In [3]:
import pandas as pd
import numpy as np

# -------------------------------
# Step 1: Load data
# -------------------------------
df = pd.read_excel("wide_data.xlsx")

# Ensure correct dtypes
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Year'].astype(int)

# -------------------------------
# Step 2: Apply selection rules
# -------------------------------

# Rule 3: For each Id-Year, pick latest rating (by date, month, day, then worst rating if tie)
df = df.sort_values(by=['Id','Year','Date'], ascending=[True, True, False])
# keep worst if same date
df = df.groupby(['Id','Year']).apply(lambda g: g.sort_values(by=['Date','Rating'], ascending=[False,False]).iloc[0]).reset_index(drop=True)

# -------------------------------
# Step 3: Handle missing years (Rule 1 & 2)
# -------------------------------

# Create full Id-Year grid
ids = df['Id'].unique()
years = range(1999, 2006)
full_index = pd.MultiIndex.from_product([ids, years], names=['Id','Year'])
df_full = pd.DataFrame(index=full_index).reset_index()

# Merge with cleaned ratings
df_full = df_full.merge(df[['Id','Year','Rating']], on=['Id','Year'], how='left')

# Fill forward previous year's rating (Rule 2)
df_full['Rating'] = df_full.groupby('Id')['Rating'].ffill()

# If first year(s) missing -> 9 (Rule 1)
df_full['Rating'] = df_full['Rating'].fillna(9)

# -------------------------------
# Step 4: Apply absorbing state (Rule 4)
# -------------------------------
def absorb(series):
    out = []
    absorbed = False
    for r in series:
        if absorbed:
            out.append(8)
        else:
            out.append(r)
            if r == 8:
                absorbed = True
    return out

df_full['Rating'] = df_full.groupby('Id')['Rating'].transform(absorb)

# -------------------------------
# Step 5: Build Transition Matrix
# -------------------------------
# For each Id-Year pair, compare Rating this year vs next year
df_full['Next_Rating'] = df_full.groupby('Id')['Rating'].shift(-1)

# Keep only 1999–2004 transitions (since 2005 has no next year)
df_tm = df_full[(df_full['Year'] >= 1999) & (df_full['Year'] < 2005)]

# Build transition counts
transitions = pd.crosstab(df_tm['Rating'], df_tm['Next_Rating'])

# Convert to probabilities row-wise
tm = transitions.div(transitions.sum(axis=1), axis=0).fillna(0)

print("Transition Matrix (1999–2005):")
print(tm.round(3))


C:\Users\swara\AppData\Local\Temp\ipykernel_10988\1831026023.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['Id','Year']).apply(lambda g: g.sort_values(by=['Date','Rating'], ascending=[False,False]).iloc[0]).reset_index(drop=True)


Transition Matrix (1999–2005):
Next_Rating    0.0    1.0    2.0    3.0    4.0    5.0    6.0    7.0    8.0  \
Rating                                                                       
0.0          0.982  0.000  0.002  0.003  0.004  0.001  0.006  0.001  0.000   
1.0          0.054  0.923  0.015  0.000  0.000  0.008  0.000  0.000  0.000   
2.0          0.033  0.012  0.885  0.068  0.001  0.000  0.001  0.000  0.000   
3.0          0.036  0.001  0.025  0.887  0.047  0.003  0.001  0.000  0.000   
4.0          0.030  0.000  0.000  0.033  0.874  0.053  0.008  0.001  0.001   
5.0          0.056  0.000  0.000  0.005  0.065  0.761  0.091  0.017  0.005   
6.0          0.055  0.000  0.002  0.003  0.006  0.067  0.785  0.072  0.009   
7.0          0.188  0.000  0.000  0.000  0.000  0.018  0.058  0.682  0.054   
8.0          0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  1.000   
9.0          0.052  0.007  0.067  0.113  0.090  0.034  0.023  0.006  0.003   

Next_Rating    9.0  
Rating     

In [5]:
# Row sums of the transition matrix
row_sums = tm.sum(axis=1)

print("Row sums:")
print(row_sums)


Row sums:
Rating
0.0    1.0
1.0    1.0
2.0    1.0
3.0    1.0
4.0    1.0
5.0    1.0
6.0    1.0
7.0    1.0
8.0    1.0
9.0    1.0
dtype: float64
